In [5]:
import xarray as xr

# open NetCDF file
data = xr.open_dataset('/media/afriesinger/Volume/Projekte/Gleitschirmfliegen/Studium/Programming/final_project/era5vis-main/era5vis/data/era5_example_dataset.nc'
)


In [6]:
data

<xarray.Dataset> Size: 27MB
Dimensions:         (valid_time: 10, pressure_level: 5, latitude: 161,
                     longitude: 281)
Coordinates:
  * valid_time      (valid_time) datetime64[ns] 80B 2025-10-01 ... 2025-10-05...
  * pressure_level  (pressure_level) float64 40B 925.0 850.0 700.0 500.0 300.0
  * latitude        (latitude) float64 1kB 70.0 69.75 69.5 ... 30.5 30.25 30.0
  * longitude       (longitude) float64 2kB -20.0 -19.75 -19.5 ... 49.75 50.0
    number          int64 8B ...
    expver          (valid_time) <U4 160B ...
Data variables:
    z               (valid_time, pressure_level, latitude, longitude) float32 9MB ...
    u               (valid_time, pressure_level, latitude, longitude) float32 9MB ...
    v               (valid_time, pressure_level, latitude, longitude) float32 9MB ...
Attributes:
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2025-11-05T10:28 GRIB to CDM+CF via cfgrib-0.9.1...